# **Data Pipeline Notebook**

### Stages of the Data Pipeline: 
1.   Environment Setup 
2.   Raw Data Processing
3.   Data Refining and Database Insertion 


Note: 
- Given the amount of time to create the project from scratch, the pipeline runs as best as it can. Given a little bit more time, most issues should be resolved. I have listed issues below:
  - There was some difficulty in running the notebooks on my local computer but I improvised and moved the work onto Google Drive and processed the data on Google Colab. There seemed to be quite a few environment problems on local computer. 
  - Relative file paths need to be used instead of absolute paths. 
  - Need to make code more robust to file name differences to make sure code can generalize well to unknown file types and different names.  
  - There was an issue trying to rearrange date format for the transaction table although code has been written to resolve the issue, it didn't seem to work in this case even after looking into documentation.
  - Exploratory data analysis should have been done on CSV files to further understand values to be inserted into the database. This would include counting the number of duplicates, anomaly detection, etc.   
- An ideal data pipeline should flow without having data be reset. This needs to be fixed although the pipeline runs as best I could make it. 

## Stage 1 - Environment Setup

In [ ]:
from google.colab import drive
drive.mount('/content/drive') # Retrieve data from Google Cloud

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install pyspark # Downloading Pyspark Library

In [20]:
import re
from datetime import datetime as dt
import datetime
from pyspark.sql.functions import *
import os
import fnmatch

# Import the SparkSession class
from pyspark.sql import SparkSession
from pyspark.sql.types import StringType, StructField, StructType, IntegerType

# Create SparkSession object
spark = SparkSession.builder \
                    .master('local[*]') \
                    .appName('Cape AI App') \
                    .getOrCreate()

## Stage 2 - Raw Data Processing

In [ ]:
%run /content/drive/MyDrive/data_engineer_assessment/Raw_Zone.ipynb

Reading customer file...
Cleaning customer file data...

First 5 rows of Customer data:
+----------+-------------------+-----------------+------------+-----------------+---------------+
|customerid|          birthdate|bank_account_type|   bank_name|employment_status|education_level|
+----------+-------------------+-----------------+------------+-----------------+---------------+
|      1000|1973-10-10 00:00:00|          Savings|Capitec Bank|             null|           null|
|      1001|1986-01-21 00:00:00|          Savings|Capitec Bank|        Permanent|           null|
|      1002|1987-04-01 00:00:00|          Savings|   Tyme Bank|             null|           null|
|      1003|1991-07-19 00:00:00|          Savings|Capitec Bank|        Permanent|           null|
|      1004|1982-11-22 00:00:00|          Savings|Capitec Bank|        Permanent|           null|
+----------+-------------------+-----------------+------------+-----------------+---------------+
only showing top 5 rows

Print

## Stage 3 - Data Refining and Database Insertion

In [25]:
%run /content/drive/MyDrive/data_engineer_assessment/Refined_Zone.ipynb

Backup performed successfully!
Data Saved as backupdatabase.sql


### Querying Database for Credit Balance of Customers

In [26]:
connection = sqlite3.connect("/content/drive/MyDrive/data_engineer_assessment/refined_zone/CustomerTransactions.db")
crsr = connection.cursor()


schema = StructType([
# Define the name field
StructField('Customer_id', IntegerType(), True),
# Add the age field
StructField('Credit_Balance', IntegerType(), True),
])

transaction_sql_command = """SELECT customer_id, SUM(transaction_amount) from factTransactions
                              group by customer_id"""

crsr.execute(transaction_sql_command)

output = crsr.fetchall()
df = spark.createDataFrame(output, schema=schema)
df.show(5)

connection.close()

+-----------+--------------+
|Customer_id|Credit_Balance|
+-----------+--------------+
|       1000|           185|
|       1003|           -31|
|       1004|          -115|
|       1005|           255|
|       1006|          -598|
+-----------+--------------+
only showing top 5 rows

